<h1><center>Laboratorio 6: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Felipe Arias
- Nombre de alumno 2: Diego Reyes


### **Link de repositorio de GitHub:** `https://github.com/dtrons/MDS7202`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- Fecha de entrega: 12/11/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

import warnings
warnings.filterwarnings('ignore')

Requirement already up-to-date: plotly in c:\users\felipe\anaconda3\lib\site-packages (5.3.1)


In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


<p align="center"><iframe width="560" height="315" src="https://www.youtube.com/embed/MW4d8_LLYp8?rel=0&amp;controls=0&amp;showinfo=0" frameborder="1" allowfullscreen></iframe></p>

# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la farbicación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito. 
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [2]:
df_retail = pd.read_pickle("online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos] 

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.
- **Recency (R)**: Indica la actualidad de la interacción de un cliente con la empresa, y da información sobre la tendencia a repetir la compra. Se define como: 
$$Recency(n)=\dfrac{1}{n} \sum^n_{i=1} date\_diff(t_{fecha final}, t_{m-i+1})$$

 Donde $date\_diff$ representa la diferencia en días entre la fecha de finalización del periodo de observación ($t_{fecha final}$), y la fecha de una visita del cliente cercana a $t_{fecha final}$, $t_{m-i+1}; t_{m}$ es la última visita del cliente; y n es el número de visitas recientes del cliente consideradas.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**Nota:** Para la $fechafinal$ utilice la fecha máxima del dataset más 1 día.

**Ejemplo de Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

In [74]:
def custom_features(dataframe_in):
    clientes=dataframe_in['Customer ID'].unique()
    df1 = pd.DataFrame(columns=['Customer ID','Length','Recency','Frequency','Monetary','Periodicity'], index=range(0))
    df1['Customer ID']=clientes
    
    length=[]
    recency=[]
    frequency=[]
    monetary=[]
    periodicity=[]
    fechafinal=df_retail['InvoiceDate'].max()
    for x in df1['Customer ID'].unique():
        a=df_retail[df_retail['Customer ID']==x].sort_values(['InvoiceDate']).reset_index()
        b=a['InvoiceDate'].min()
        c=a['InvoiceDate'].max()
        length.append((c-b).days)
        recency.append((fechafinal-c).days+1)
        frequency.append(len(a))
        visitas=a['InvoiceDate'].map(lambda t: t.date()).unique()
        monetary.append(a['Price'].sum()/len(visitas))
        dif=[]
        for e in range(len(visitas)-1):
            dif.append((visitas[e+1]-visitas[e]).days)
        periodicity.append(np.std(dif))
    
    df1['Length']=length
    df1['Recency']=recency
    df1['Frequency']=frequency
    df1['Periodicity']=periodicity
    df1['Monetary']=monetary
    
    df1=df1[df1['Length']>1]
    
    return df1
    

**Resúesta de Pauta:**

## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [99]:
class MinMax(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
        
    
    def fit(self, X, y=None):
        return self
    
    def aux(self, x):
        return (x-x.min())/(x.max()-x.min())
    
    def transform(self, X):
        data_copy=X.copy()
        data_aux=data_copy.select_dtypes([np.number])
        data_aux.transform(self.aux)
        #col=list(data_aux.columns)
        #data_copy[col]=data_aux[col]
        return data_aux

In [100]:
my_tf = MinMax()
my_tf.transform(lrmfp)

,Length,Recency,Frequency,Monetary,Periodicity
0,0.153639,0.857923,0.010780,0.041681,0.151934
1,0.997305,0.002732,0.079231,0.032445,0.031142
2,0.776280,0.226776,0.006827,0.042806,0.000000
3,1.000000,0.000000,0.112289,0.030115,0.046494
4,1.000000,0.000000,0.091987,0.045482,0.057789
...,...,...,...,...,...
4242,0.018868,0.002732,0.015271,0.061543,0.000000
4246,0.018868,0.002732,0.018326,0.081057,0.000000
4251,0.021563,0.000000,0.006648,0.041958,0.000000
4259,0.010782,0.008197,0.008803,0.071084,0.000000


### 1.3.2 `T-SNA` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo. 

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior. 
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo T-sne de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [102]:

# Creamos una instancia de tsne
tsne = TSNE(n_components=2, random_state=42)


LRMFP = FunctionTransformer(custom_features)
ct = ColumnTransformer([
    ("MinMax", MinMax(), ['Length','Recency','Frequency','Monetary','Periodicity'])],
    remainder='passthrough')

In [103]:
pipeline = Pipeline([('LRMFP', LRMFP),('Preprocesamiento',ct),('tsne',tsne)])

In [106]:
retail_features_tsne_embedded = pipeline.fit_transform(df_retail)
retail_features_tsne_embedded

C:\Users\Felipe\Anaconda3\lib\site-packages\numpy\core\_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\Felipe\Anaconda3\lib\site-packages\numpy\core\_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\Felipe\Anaconda3\lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


array([[ 16.597115 , -58.616146 ],
       [  3.1707366, -40.793934 ],
       [ 17.429482 ,  50.33777  ],
       ...,
       [-50.457073 , -24.192266 ],
       [-52.181053 ,  -5.9908266],
       [-52.144054 ,  -5.5918612]], dtype=float32)

In [108]:
df=LRMFP.transform(df_retail)

C:\Users\Felipe\Anaconda3\lib\site-packages\numpy\core\_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\Felipe\Anaconda3\lib\site-packages\numpy\core\_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\Felipe\Anaconda3\lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [109]:
df

,Customer ID,Length,Recency,Frequency,Monetary,Periodicity
0,13085.0,59,315,62,70.350000,27.500000
1,13078.0,372,2,443,54.831290,5.636784
2,15362.0,290,84,40,72.240000,0.000000
3,18102.0,373,1,627,50.917209,8.415412
4,12682.0,373,1,514,76.734783,10.459781
...,...,...,...,...,...,...
4242,16893.0,9,2,87,103.720000,0.000000
4246,17722.0,9,2,104,136.505000,0.000000
4251,17671.0,10,1,39,70.815000,0.000000
4259,17223.0,6,4,51,119.750000,0.000000


In [110]:
df['x_tsne'] = retail_features_tsne_embedded[:, 0]
df['y_tsne'] = retail_features_tsne_embedded[:, 1]

In [114]:
px.scatter(df, x='x_tsne', y='y_tsne')

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero permutando el algoritmo de reducción de dimensionalidad por k-means.

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [153]:
intertias = []


for i in range(1, 20):
    kmeans=KMeans(n_clusters=i, random_state=0)
    pipeline_k = Pipeline([('LRMFP', LRMFP),('Preprocesamiento',ct),('kmeans',kmeans)])
    pipeline_k.fit_transform(df_retail)
    intertias.append([i,pipeline_k.named_steps['kmeans'].inertia_])
    
intertias = pd.DataFrame(intertias, columns=["n° clusters", "inertia"])
intertias.head(10)

,n° clusters,inertia
0,1,8.157005e+09
1,2,2.185790e+09
2,3,1.125007e+09
3,4,7.230560e+08
4,5,5.475717e+08
5,6,4.478750e+08
6,7,3.856665e+08
7,8,3.256244e+08
8,9,2.899059e+08
9,10,2.640116e+08


In [154]:
px.line(intertias, x="n° clusters", y="inertia", title="Método del Codo con K-Means")

Como se puede apreciar en el grafico anterior, segun el metodo del codo se tiene el punto el numero de cluster escogido corresponde a 6, ya que desde ese punto los cambios en la inercia no son de gran tamaño.

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [155]:
kmeans=KMeans(n_clusters=6, random_state=0)
pipeline_k = Pipeline([('LRMFP', LRMFP),('Preprocesamiento',ct),('kmeans',kmeans)])


In [156]:
pipeline_k.fit_transform(df_retail)

array([[4689.18435645, 1780.44398665, 2794.05145485,  824.42668825,
         372.34956999, 3758.18655938],
       [4699.49855049, 1792.45191427, 2809.50891039,  858.58927465,
         419.12981978, 3770.55678644],
       [2405.93389361,  543.20274343,  516.27011409, 1513.33594441,
        2500.12500376, 1474.47289173],
       ...,
       [ 246.25155588, 2842.91812412, 1820.96255236, 3824.75679858,
        4810.68999066,  869.13958606],
       [ 588.80457683, 2396.87744379, 1377.65608724, 3377.91089719,
        4363.28874886,  452.40566732],
       [ 609.07858448, 2371.14656686, 1352.07499531, 3352.63696348,
        4337.97252035,  427.66635001]])

In [157]:
pipeline_k.named_steps['kmeans'].labels_

array([4, 4, 2, ..., 0, 5, 5])

In [158]:
df_k=LRMFP.transform(df_retail)

In [159]:
df_k['cluster']=pipeline_k.named_steps['kmeans'].labels_

In [161]:
data_aux=df_k.select_dtypes([np.number])

In [165]:
data_aux

,Length,Recency,Frequency,Monetary,Periodicity,cluster
0,59,315,62,70.350000,27.500000,4
1,372,2,443,54.831290,5.636784,4
2,290,84,40,72.240000,0.000000,2
3,373,1,627,50.917209,8.415412,0
4,373,1,514,76.734783,10.459781,4
...,...,...,...,...,...,...
4242,9,2,87,103.720000,0.000000,5
4246,9,2,104,136.505000,0.000000,0
4251,10,1,39,70.815000,0.000000,0
4259,6,4,51,119.750000,0.000000,5


In [167]:
data_aux[data_aux['Periodicity']>0].groupby('cluster').agg('mean')

,Length,Recency,Frequency,Monetary,Periodicity
cluster,,,,,
0,249.389241,42.829114,178.727848,67.040375,34.285407
1,242.005865,43.739003,190.782991,75.998553,33.947646
2,232.653295,47.068768,146.896848,74.424596,33.225659
3,250.857143,51.108571,147.474286,76.053847,34.193988
4,241.870432,46.996678,171.259136,80.681478,29.512745
5,247.476038,41.750799,165.364217,75.620900,34.779609


Como se puede apreciar no hay una gran diferencia de manera significativa en los features generados de los clientes es estos 6 grupos generados, pero al ir en detalle por ejemplo se puede aprecuar que el cluster numero 2 corresponden a clientes que tienen una menor frecuencia al momento de comprar, pero si una mayor cantidad de compras, por lo que podria corresponder a clientes que hacen compras mensuales lo cual, compran mucho pero a la vez van poco al supermercado.

Por otro lado se tiene por ejemplo el cluster numero 4, el cual corresponde a los clientes que gastan mas al comprar y tienen el menor periodicity, es decir, que estan constantemente comprando en intervalos de tiempo menor a los demas tipos de clientes.

**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [169]:
df_3=df.copy()
df_3['cluster']=pipeline_k.named_steps['kmeans'].labels_
df_3

,Customer ID,Length,Recency,Frequency,Monetary,Periodicity,x_tsne,y_tsne,cluster
0,13085.0,59,315,62,70.350000,27.500000,16.597115,-58.616146,4
1,13078.0,372,2,443,54.831290,5.636784,3.170737,-40.793934,4
2,15362.0,290,84,40,72.240000,0.000000,17.429482,50.337769,2
3,18102.0,373,1,627,50.917209,8.415412,-26.729120,-22.239576,0
4,12682.0,373,1,514,76.734783,10.459781,-7.374687,-57.923416,4
...,...,...,...,...,...,...,...,...,...
4242,16893.0,9,2,87,103.720000,0.000000,-53.344208,13.302183,5
4246,17722.0,9,2,104,136.505000,0.000000,-48.039570,-29.306040,0
4251,17671.0,10,1,39,70.815000,0.000000,-50.457073,-24.192266,0
4259,17223.0,6,4,51,119.750000,0.000000,-52.181053,-5.990827,5


In [171]:
px.scatter(df_3, x='x_tsne', y='y_tsne',color='cluster')

Como se puede apreciar, si separa bien los 6 cluster generados con las 2 dimensiones generadas con t-sne.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab** solo para tareas. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>